# CTA "L" accessibility analysis


In [1069]:
import math
import itertools
from pathlib import Path
import numpy as np
import pandas as pd
from matplotlib import pyplot

In [1070]:
STATION_DATA_PATH = Path(
    "data/CTA_-_System_Information_-_List_of__L__Stops_20240921.csv")

In [1071]:
stations_df = pd.read_csv(STATION_DATA_PATH,
                          usecols=[
                              "MAP_ID", "STATION_NAME", "STATION_DESCRIPTIVE_NAME", "ADA"],
                          dtype={"DIRECTION_ID": "category",
                                 "STOP_NAME": pd.StringDtype(),
                                 "STATION_NAME": pd.StringDtype(),
                                 "STATION_DESCRIPTIVE_NAME": pd.StringDtype(), },

                          )

In [1072]:

RIDERSHIP_DATA_PATH = Path(
    "data/CTA_-_Ridership_-__L__Station_Entries_-_Monthly_Day-Type_Averages___Totals_20240921.csv")

In [1073]:
ridership_df = pd.read_csv(
    RIDERSHIP_DATA_PATH,
    usecols=["station_id", "stationame", "month_beginning", "monthtotal"],
    dtype={
        "stationname": pd.StringDtype()
    },
    parse_dates=["month_beginning"],
)

## Picking our dates


In [1074]:
ridership_by_date = ridership_df.set_index("month_beginning").sort_index()
ridership_by_date

,station_id,stationame,monthtotal
month_beginning,,,
2001-01-01,40900,Howard,164447
2001-01-01,40510,Garfield-South Elevated,19359
2001-01-01,40130,51st,29609
2001-01-01,41080,47th-South Elevated,32826
2001-01-01,41270,43rd,19871
...,...,...,...
2024-05-01,41330,Montrose-O'Hare,41468
2024-05-01,40550,Irving Park-O'Hare,72592
2024-05-01,41240,Addison-O'Hare,49949


It looks like May 2024 is the last month for which ridership numbers are avilable in the data that I have. The station data was last updated more recently, in August 2024. I think it's fine to start with the latest ridership data. Let us isolate that.


In [1075]:
latest_ridership_df = ridership_by_date[ridership_by_date.index >= pd.Timestamp(
    '2024-05-01')]
latest_ridership_df

,station_id,stationame,monthtotal
month_beginning,,,
2024-05-01,40480,Cicero-Lake,21842
2024-05-01,40030,Pulaski-Lake,25981
2024-05-01,41670,Conservatory,17928
2024-05-01,41070,Kedzie-Lake,19456
2024-05-01,41360,California-Lake,19507
...,...,...,...
2024-05-01,41330,Montrose-O'Hare,41468
2024-05-01,40550,Irving Park-O'Hare,72592
2024-05-01,41240,Addison-O'Hare,49949


Let's quickly check that each station ID only occurs once


In [1076]:
station_value_counts = latest_ridership_df.value_counts("station_id")
station_value_counts

station_id
40010    1
40020    1
40030    1
40040    1
40050    1
        ..
41660    1
41670    1
41680    1
41690    1
41700    1
Name: count, Length: 143, dtype: int64

In [1077]:
station_value_counts.max()

np.int64(1)

Now we need to join the ridership information with the accessibility information.


First, for convenience, create the smallest possible table containing only station IDs and ADA information. As explained below, every stataion is either fully accessible or fully inaccessible according to the data so we can safely drop duplicates and know that each station ID will only appear once. But I check below anyway because I'm overly cautious.


In [1078]:
access_df = stations_df[["MAP_ID", "ADA"]].drop_duplicates()
access_df

,MAP_ID,ADA
0,40420,True
1,40780,True
2,40940,True
3,40230,True
4,40470,False
...,...,...
221,41440,True
253,40260,False
254,40480,True
255,41330,False


In [1079]:
access_df.value_counts("MAP_ID").describe()[["min", "max"]]

min    1.0
max    1.0
Name: count, dtype: float64

Now we can join the accessibility to the ridership data.


In [1080]:
access_df = access_df
access_df

,MAP_ID,ADA
0,40420,True
1,40780,True
2,40940,True
3,40230,True
4,40470,False
...,...,...
221,41440,True
253,40260,False
254,40480,True
255,41330,False


In [1081]:
accessible_ridership_df = latest_ridership_df.merge(
    access_df, how="left", left_on="station_id", right_on="MAP_ID")
accessible_ridership_df

,station_id,stationame,monthtotal,MAP_ID,ADA
0,40480,Cicero-Lake,21842,40480,True
1,40030,Pulaski-Lake,25981,40030,True
2,41670,Conservatory,17928,41670,True
3,41070,Kedzie-Lake,19456,41070,True
4,41360,California-Lake,19507,41360,True
...,...,...,...,...,...
138,41330,Montrose-O'Hare,41468,41330,False
139,40550,Irving Park-O'Hare,72592,40550,False
140,41240,Addison-O'Hare,49949,41240,True
141,40970,Cicero-Forest Park,16943,40970,False


Now check to make sure there aren't any NaNs after the join.


In [1082]:
accessible_ridership_df.isna().sum()

station_id    0
stationame    0
monthtotal    0
MAP_ID        0
ADA           0
dtype: int64

## How many accessible stations are there?


There are 101 accessible stations and 42 inaccessible stations with ridership data in the latest month of available data, May 2024. That means 70% of stations are accessible and 30% are not.


In [1083]:
accessible_ridership_df.value_counts("ADA")

ADA
True     101
False     42
Name: count, dtype: int64

## How many trips are there?


There are 9,585,547 trips (turnstyle entries) reported in the month of May 2024.


In [1084]:
accessible_ridership_df[["monthtotal"]].sum()

monthtotal    9585547
dtype: int64

## What proportion of trips begin at accessible stations?


First, let's try breaking down the monthly total by ADA designation and make sure the total matches the total trips above.

In [1085]:
accessible_trips_bulk = accessible_ridership_df[[
    "monthtotal", "ADA"]].groupby("ADA").sum()
accessible_trips_bulk

,monthtotal
ADA,
False,2324456
True,7261091


In [1086]:
accessible_Trips_bulk_sum = accessible_trips_bulk.sum()
accessible_Trips_bulk_sum

monthtotal    9585547
dtype: int64

In [1087]:
accessible_Trips_bulk_sum == accessible_ridership_df[["monthtotal"]].sum()

monthtotal    True
dtype: bool

The total rides match after being divided among accessible and inaccessible trip starts.  So now let's convert those bulk totals into proportions.

In [1088]:
accessible_trips_proportion = accessible_trips_bulk
accessible_trips_proportion["prop"] = accessible_trips_proportion['monthtotal'] / \
    accessible_trips_proportion['monthtotal'].sum()
accessible_trips_proportion

,monthtotal,prop
ADA,,
False,2324456,0.242496
True,7261091,0.757504


Make sure that the proportions add to one.

In [1089]:
accessible_trips_proportion['prop'].sum()

np.float64(1.0)

So, about 76% "L" trips in May 2024 started at accessible stations, and about 24% started at inaccessible stations.

Now we know where we are, and we need to compare that to where we could be.

## Datasets


1.  [List of 'L' stops](https://data.cityofchicago.org/Transportation/CTA-System-Information-List-of-L-Stops/8pix-ypme/about_data)
2.  [CTA - Ridership - 'L' Station Entries - Monthly Day-Type Averages & Totals](https://data.cityofchicago.org/Transportation/CTA-Ridership-L-Station-Entries-Monthly-Day-Type-A/t2rn-p8d7/about_data)


### Findings up front


1.  The columns that tie the datasets together are "MAP_ID" in the list of "L" stops and "station_id" in the ridership data.
2.  There are no stations with both accessible and non-accesible stops.


### List of 'L' stops


Source: [List of 'L' stops](https://data.cityofchicago.org/Transportation/CTA-System-Information-List-of-L-Stops/8pix-ypme/about_data)


#### Overview


The dataset is about stops. Stops include details such as direction of travel. Each station therefore has at least two stops. For example, one for northbound trains and one for southbound trains. Station and stop IDs are not directly related. Instead, the "MAP_ID" column references the stop's parent station.

ADA accessibility information is stored in a boolean column named "ADA". Because there is a row for each stop and multiple stops per station the structure of the data technically allows for stations that are partially accessible. If such mixed-accessibility stations actually existed in the data then choices would need to be made about how to handle ridership data from them. Thankfully they do not occur in the data.

The various name columns, espeically "STATION_NAME" are not unique. There are multiple stations with the same name, even on the same line. The blue line has two stops named Western and two named Harlem, for example.

There are are boolean columns indicating which lines a stop serves, and a location colum with physical location data. Since these fields are not needed to uniquely identify a station, we don't need them for our analysis.


In [1090]:
stations_df.head()

,STATION_NAME,STATION_DESCRIPTIVE_NAME,MAP_ID,ADA
0,Cicero,Cicero (Pink Line),40420,True
1,Central Park,Central Park (Pink Line),40780,True
2,Halsted,Halsted (Green Line),40940,True
3,Cumberland,Cumberland (Blue Line),40230,True
4,Racine,Racine (Blue Line),40470,False


### Ridership data


Source: [CTA - Ridership - 'L' Station Entries - Monthly Day-Type Averages & Totals](https://data.cityofchicago.org/Transportation/CTA-Ridership-L-Station-Entries-Monthly-Day-Type-A/t2rn-p8d7/about_data)


#### Overview


This dataset is about stations. Specifically, it tracks turnstyle numbers. These numbers tell us nothing about which direction a passenger intends to travel and therefore nothing abou the stop that they intend to use. This means that there is a mismatch in the level of detail between the dataset describing how many people ride and the dataset describing where they can go.

The "station_id" column is the shared key between the datasets. As in the list of stops, the "stationame" column is not unique.


In [1091]:
ridership_df.head()

,station_id,stationame,month_beginning,monthtotal
0,40900,Howard,2001-01-01,164447
1,41190,Jarvis,2001-01-01,40567
2,40100,Morse,2001-01-01,119772
3,41300,Loyola,2001-01-01,125008
4,40760,Granville,2001-01-01,84189


### Demonstrating the shared key between datasets


The datasets are not described in great detail, so some of the relationships must be demonstrated through examination. There's no obvious reason why a column named "MAP_ID" would identify a station, for example. Let us check some examples.


First we need to find some station identifiers. I know that there are some stops that contain the word "Lake" so let's get those.


In [1092]:
stations_df[stations_df["STATION_DESCRIPTIVE_NAME"].str.contains("Lake")]

,STATION_NAME,STATION_DESCRIPTIVE_NAME,MAP_ID,ADA
7,Clark/Lake,"Clark/Lake (Blue, Brown, Green, Orange, Purple...",40380,True
36,Lake,Lake (Red Line),41660,True
56,Harlem/Lake,Harlem/Lake (Green Line),40020,True
62,Clark/Lake,"Clark/Lake (Blue, Brown, Green, Orange, Purple...",40380,True
103,Lake,Lake (Red Line),41660,True
180,Clark/Lake,"Clark/Lake (Blue, Brown, Green, Orange, Purple...",40380,True
253,State/Lake,"State/Lake (Brown, Green, Orange, Pink & Purpl...",40260,False
261,Harlem/Lake,Harlem/Lake (Green Line),40020,True
267,State/Lake,"State/Lake (Brown, Green, Orange, Pink & Purpl...",40260,False
290,Clark/Lake,"Clark/Lake (Blue, Brown, Green, Orange, Purple...",40380,True


Let's try searching the ridership data for a few of the station identifiers we just found and see if they match.


In [1093]:
ridership_df[ridership_df["station_id"] == 40380].value_counts("stationame")

stationame
Clark/Lake    281
Name: count, dtype: int64

In [1094]:
ridership_df[ridership_df["station_id"] == 41660].value_counts("stationame")

stationame
Lake/State    281
Name: count, dtype: int64

In [1095]:
ridership_df[ridership_df["station_id"] == 40020].value_counts("stationame")

stationame
Harlem-Lake    281
Name: count, dtype: int64

As we can see from the examples above the short names for the stations match - to a human - when linked by the ID columns that we found.


The datasets are clearly not carefully groomed since even the same dataset uses slashes to separate intersection names sometimes (Lake/State) and uses dashes (Harlem-Lake) at other times. But we already knew this becuase "MAP_ID" and "station_id" are not in the same format. That's just the nature of real-world datasets.


### Finding orphans


Ideally the mapping between stations and ridership data would be complete. Every station would have ridership numbers and all the ridership info would be linked to a station we knew about. Unfortunately that is not the case. The code below finds any station identifiers that only occur in one dataset or the other. There are five of them. They are explained below.


The ridership data with no associated station information is for stations that have been closed. The station with no ridership information is too new to appear in the data. A major source of this information was the following link: https://www.chicago-l.org/stations/index.html


1.  Madison/Wabash was closed in 2015
2.  Washington/State has been temporarily closed since 2006
3.  Randolph/Wabash was closed in 2017.
4.  Homan has been closed since 1994, permanently removed from the line in 1996
5.  The Damen green line station opened in August 20224 so ridership data is not yet available in September 2024.


Stations with no ridership data:

1.  41710: Damen (Green Line)


Ridership data with no station information:

1.  40500: Washington/State
2.  40640: Madison/Wabash
3.  40200: Randolph/Wabash
4.  41580: Homan


In [1096]:
def find_orphan_station_ids(stations=stations_df, ridership=ridership_df):
    map_id_set = set(stations["MAP_ID"])
    station_id_set = set(ridership["station_id"])
    print(f"There are {len(map_id_set)} unique entries in the MAP_ID column")
    print(f"There are {len(station_id_set)
                       } unique entries in the station_id column")
    print(f"There are {len((sd := map_id_set.symmetric_difference(
        station_id_set)))} entries that only appear in one column or the other")
    print(f"They are: {sd}")
    return sd

In [1097]:
orphans = find_orphan_station_ids()

There are 144 unique entries in the MAP_ID column
There are 147 unique entries in the station_id column
There are 5 entries that only appear in one column or the other
They are: {40500, 41580, 40640, 41710, 40200}


In [1098]:
orphans_in_stop_list = stations_df[stations_df['MAP_ID'].isin(orphans)]
print(orphans_in_stop_list)

    STATION_NAME STATION_DESCRIPTIVE_NAME  MAP_ID   ADA
300        Damen       Damen (Green Line)   41710  True
301        Damen       Damen (Green Line)   41710  True


In [1099]:
oprhans_in_ridership_data = ridership_df[ridership_df["station_id"].isin(
    orphans)][["station_id", "stationame"]].drop_duplicates()
print(oprhans_in_ridership_data)

     station_id        stationame
20        40500  Washington/State
138       40640    Madison/Wabash
139       40200   Randolph/Wabash
945       41580             Homan


### Looking for mixed stations


As I wrote in the descriptions of the datasets, ADA accessibility is tracked by stop. Each station has multiple stops, which means that the format of the data allows for stations that are only partially accessible. Since the ridership data only tracks entry into stations, partially accessible staitons would lead to ambiguity. We woudln't know how many of the passengers used the accessible stops. I also said that no such stations exist. Let me demonstrate that.


First, let us obtian the count of accessible and inaccessible stops at each station.


In [1100]:
trimmed_station_data = stations_df[[
    "MAP_ID", "STATION_DESCRIPTIVE_NAME", "ADA"]]
station_vcs = trimmed_station_data.value_counts()
station_accessability_counts = station_vcs.unstack()
station_accessability_counts

,ADA,False,True
MAP_ID,STATION_DESCRIPTIVE_NAME,,
40010,Austin (Blue Line),2.0,NaN
40020,Harlem/Lake (Green Line),NaN,2.0
40030,Pulaski (Green Line),NaN,2.0
40040,"Quincy/Wells (Brown, Orange, Purple & Pink lines)",NaN,2.0
40050,Davis (Purple Line),NaN,2.0
...,...,...,...
41670,Conservatory (Green Line),NaN,2.0
41680,Oakton-Skokie (Yellow Line),NaN,2.0
41690,Cermak-McCormick Place (Green Line),NaN,2.0


The table above counts the number of times True and False occur in the ADA column for each station. If a station never appears with a particular value in the ADA column, the table shows NaN. Usually I'd replace those NaNs with zeroes, but in this case NaNs are very convenient. Any partially accessible stations, and only partially accessible stations, would have stops where ADA column was false and stops where ADA column was true. Dropping rows with NaNs will eliminate stations that are fully accessible or fully inaccessible, leaving only the stations wer're looking for.


In [1101]:
station_accessability_counts.dropna()

,ADA,False,True
MAP_ID,STATION_DESCRIPTIVE_NAME,,


There are no stations in the data with both accessible and inaccessible stops.
